In [1]:
import sys
sys.path.append("../")

In [2]:
import numpy as np
from experiment_analysis import analyze_and_plot_features

# Server 

In [ ]:
# Define fixed parameters that do not change during the analysis
fixed_params = {
    'countries': ['US', 'UK', 'DE', 'FR', 'CA', 'AU', 'JP', 'IN'],  # List of countries to be analyzed
    'platforms': ['iOS', 'Android', 'Web', 'Desktop'],  # Platforms to be included in the analysis
    'user_segments': ['Segment_1', 'Segment_2', 'Segment_3', 'Segment_4'],  # User segments to be considered
    'ab_groups': ['a1', 'b'],  # A/B test groups
}

# Define parameters with ranges
varying_params = {
    'base_increase_percentage': np.arange(0.0, 0.1, 0.01),  # Range of base increase percentages to be analyzed
    'noise_level': np.arange(0.0, 1, 0.1),
    'alpha': np.arange(0.0, 1, 0.1),
}

# Define parameters with ranges to be used as the X-axis
x_params = {
    'num_users': np.arange(10, 6000, 200)  # Range of user numbers to be used on the X-axis
}

# Call the function to analyze and plot for all selected features
analyze_and_plot_features(
    fixed_params, 
    varying_params, 
    x_params, 
    num_iterations=10,  # Number of iterations to perform for each analysis
)
